In [13]:
from qiskit import Aer, QuantumCircuit, transpile, assemble,IBMQ
from qiskit.visualization import plot_histogram
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
import numpy as np

# Define the 3SAT instance
input_3sat_instance = '''
c example DIMACS-CNF 3-SAT
p cnf 3 5
-1 -2 -3 0
1 -2 3 0
1 2 -3 0
1 -2 -3 0
-1 -2 3 0
'''

# Function to convert 3SAT instance to quantum circuit
def sat_to_circuit(instance):
    clauses = instance.strip().split('\n')[2:]
    num_qubits = 3  # Number of qubits needed for this problem

    # Create a quantum circuit with num_qubits qubits
    qc = QuantumCircuit(num_qubits, num_qubits)

    # Encode each clause into quantum gates
    for clause in clauses:
        literals = list(map(int, clause.split()[:-1]))
        qubit_indices = [abs(literal) - 1 for literal in literals]
        
        # Apply the OR gate for each clause
        qc.or_gate(qubit_indices)

    # Measure the qubits
    qc.measure(range(num_qubits), range(num_qubits))

    return qc

# Custom function to implement OR gate
def or_gate(self, qubit_indices):
    for index in qubit_indices[:-1]:
        self.cx(index, qubit_indices[-1])  # Apply CX (CNOT) gate
    return self

# Add custom OR gate function to QuantumCircuit class
QuantumCircuit.or_gate = or_gate

# Convert 3SAT instance to quantum circuit
qc = sat_to_circuit(input_3sat_instance)

# Simulate the circuit
simulator = Aer.get_backend('qasm_simulator')
transpiled_qc = transpile(qc, simulator)
qobj = assemble(transpiled_qc,shots=1000)
result = simulator.run(qobj).result()
counts = result.get_counts()

# Display the result
print(counts)


{'000': 1000}


C:\Users\Sakhawat Shah\AppData\Local\Temp\ipykernel_10908\109155077.py:55: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = simulator.run(qobj).result()


In [14]:
# Load IBM Quantum account
provider = IBMQ.load_account()

# Get the least busy backend
backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 3 and
                                   not x.configuration().simulator and x.status().operational==True))

print("Least busy backend:", backend)

# Convert 3SAT instance to quantum circuit
qc = sat_to_circuit(input_3sat_instance)

# Transpile the circuit for the backend
transpiled_qc = transpile(qc, backend)

# Submit job to the backend
job = backend.run(transpiled_qc)

# Monitor the job
job_monitor(job)

# Get the result
result = job.result()
counts = result.get_counts()

# Display the result
print(counts)

C:\Users\Sakhawat Shah\AppData\Local\Temp\ipykernel_10908\1369724046.py:2: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  provider = IBMQ.load_account()


Least busy backend: ibm_brisbane
Job Status: job has successfully run
{'011': 4, '000': 3355, '110': 153, '010': 152, '111': 7, '100': 232, '001': 41, '101': 56}
